In [ ]:
!pip install catboost

In [ ]:
import numpy as np
import pandas as pd

import datetime
from pandas_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier,\
    Perceptron, RidgeClassifier, SGDClassifier, Ridge
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, Pool, CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import cross_val_score
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv('../input/real-time-advertisers-auction/Dataset.csv')
print(data.shape)
data.head()

In [ ]:
# calculating CPM
# calculating the value that the Advertisers Bid for the month of June
# CPM(the value which was the winning bid value) =
# ((revenue of the publisher * 100) / revenue_share_percentage) /\
# measurable_impressions) * 1000


def weird_division(n, d):
    return n / d if d else 0


data['CPM'] = data.apply(lambda x: weird_division(
    ((x['total_revenue'] * 100)), x['measurable_impressions']) * 1000, axis=1)

In [ ]:
negative_cpm = list(data[data['CPM'] < 0].index)
negative_cpm

In [ ]:
data_date_time = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')
split_flag = datetime.datetime.strptime('21:06:2019', '%d:%m:%Y')

In [ ]:
train_indexes = list(data_date_time[data_date_time <= split_flag].index)
test_indexes = data.loc[data_date_time[data_date_time > split_flag].index, :]['CPM'][data.loc[data_date_time[
    data_date_time > split_flag].index, :]['CPM'] < np.percentile(data.loc[
    data_date_time[data_date_time > split_flag].index, :]['CPM'], 95)].index
test_indexes = list(test_indexes)
for i in negative_cpm:
    if i in test_indexes:
        test_indexes.remove(i)

In [ ]:
print(f'Величина train равна {len(train_indexes)}')
print(f'Величина test равна {len(test_indexes)}')

In [ ]:
# %%time
# for i in train_indexes:
#     if i in test_indexes:
#         print('FAIL TRAIN')
#         break
# for i in test_indexes:
#     if i in train_indexes:
#         print('FAIL TEST')
#         break
# 28 min +

In [ ]:
train_data = data.loc[train_indexes, :].drop(['total_revenue'], axis=1)
test_data = data.loc[test_indexes, :].drop(['total_revenue', 'CPM'], axis=1)
total_target = data.loc[test_indexes, :]['CPM']
train_data.shape, test_data.shape, len(total_target)

In [ ]:
# попробуем поменьше отрезать у трейна
train_data = train_data[train_data['CPM'] <= np.percentile(train_data['CPM'], 96)]

# Начинаем анализ train

In [ ]:
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
train_data.describe(include=[np.object])

In [ ]:
data.isnull().sum()

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
# %%time
# profile = ProfileReport(train_data)
# profile.to_file(output_file='train_data_CPM.html')
# больше часа

In [ ]:
# %%time
# profile = ProfileReport(train_data, minimal=True)
# profile.to_file(output_file='train_data_CPM.html')


# date

In [ ]:
train_data['date'] = pd.to_datetime(train_data['date'], format='%Y-%m-%d %H:%M:%S')
test_data['date'] = pd.to_datetime(test_data['date'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
dict_for_lebel_date = {}
for i, date in enumerate(sorted(pd.concat([train_data['date'], test_data['date']]).unique())):
    dict_for_lebel_date[date] = i
dict_for_lebel_date

In [ ]:
train_data['date'].replace(dict_for_lebel_date, inplace=True)
test_data['date'].replace(dict_for_lebel_date, inplace=True)

In [ ]:
train_data['date'].value_counts()[list(range(len(train_data['date'].value_counts())))]

In [ ]:
f, ax = plt.subplots(figsize=(12, 5))
sns.lineplot(x=range(len(train_data['date'].value_counts())),
             y=train_data['date'].value_counts()[sorted(train_data['date'].value_counts().index)])
ax.set_title('Количество кейсов в train', fontdict={'size':16})
ax.set(xlabel='время')
ax.set(ylabel='количество')

In [ ]:
f, ax = plt.subplots(figsize=(12, 5))
sns.lineplot(x=range(len(test_data['date'].value_counts())),
             y=test_data['date'].value_counts()[sorted(test_data['date'].value_counts().index)])
ax.set_title('Количество кейсов в test', fontdict={'size':16})
ax.set(xlabel='время')
ax.set(ylabel='количество')

In [ ]:
f, ax = plt.subplots(figsize=(12, 5))
sns.lineplot(x=train_data.groupby(by=['date']).mean()['CPM'].index,
             y=train_data.groupby(by=['date']).mean()['CPM'])
ax.set_title('Среднее CPM в train', fontdict={'size':16})
ax.set(xlabel='время')
ax.set(ylabel='CPM')

In [ ]:
f, ax = plt.subplots(figsize=(12, 5))
sns.lineplot(x=train_data.groupby(by=['date']).median()['CPM'].index,
             y=train_data.groupby(by=['date']).median()['CPM'])
ax.set_title('Медиана CPM в train', fontdict={'size':16})
ax.set(xlabel='время')
ax.set(ylabel='CPM')

Надо дальше смотреть связь по датам, может получится выделить пачку временных рядов (upd, не получилось выделить)

# site_id 

In [ ]:
train_data['site_id'].value_counts()

### site_id возьмём как обычный кат-признак

# ad_type_id

In [ ]:
train_data['ad_type_id'].value_counts()

In [ ]:
train_data.groupby(by='ad_type_id').describe()['CPM']

### ad_type_id  
2 значения, в отличаются перекосами и количеством нулей сильно, оставим меньший тоже

# geo_id

In [ ]:
train_data['geo_id'].value_counts()

In [ ]:
flag_value = 100
print(f"Такое количесто датасета оставим: \
{train_data['geo_id'].value_counts()[train_data['geo_id'].value_counts() > flag_value].sum() / train_data.shape[0] * 100}\
%")
display(train_data['geo_id'].value_counts()[train_data['geo_id'].value_counts() > flag_value])

In [ ]:
train_data = train_data[train_data['geo_id'].isin(train_data['geo_id'].value_counts()[
    train_data['geo_id'].value_counts() > flag_value].index)]

### geo_id убрали меньше процента выбросов

# device_category_id

In [ ]:
def display_value_counts_and_norm(obj):
    display(obj.value_counts())
    display(obj.value_counts(normalize=True))

In [ ]:
display_value_counts_and_norm(train_data['device_category_id'])

In [ ]:
train_data = train_data[train_data['device_category_id'].isin([2, 1, 3])]

### device_category_id убрали выбросы

# advertiser_id

In [ ]:
display_value_counts_and_norm(train_data['advertiser_id'])

In [ ]:
train_data = train_data[train_data['advertiser_id'].isin(train_data['advertiser_id'].value_counts().index[:-6])]

### advertiser_id тоже убрали выбросы

# order_id

In [ ]:
display_value_counts_and_norm(train_data['order_id'])

In [ ]:
train_data['order_id'].value_counts(normalize=True)[train_data['order_id'].value_counts(normalize=True) < 0.0018].sum()

In [ ]:
train_data = train_data[train_data['order_id'].isin(train_data['order_id'].value_counts(
    normalize=True)[train_data['order_id'].value_counts(normalize=True) >= 0.0018].index)]

### order_id тоже удалили процент выбросов

# line_item_type_id

In [ ]:
display_value_counts_and_norm(train_data['line_item_type_id'])

# os_id

In [ ]:
display_value_counts_and_norm(train_data['os_id'])

In [ ]:
train_data = train_data[~train_data['os_id'].isin([15, 57])]

### os_id

# integration_type_id

In [ ]:
display_value_counts_and_norm(train_data['integration_type_id'])

In [ ]:
train_data.drop(['integration_type_id'], axis=1, inplace=True)
test_data.drop(['integration_type_id'], axis=1, inplace=True)

### дропнули константу

# monetization_channel_id

In [ ]:
display_value_counts_and_norm(train_data['monetization_channel_id'])

### здесь и далее делаем одно и то же, отбрасываем выбросы, если есть

# ad_unit_id

In [ ]:
display_value_counts_and_norm(train_data['ad_unit_id'])

In [ ]:
train_data = train_data[train_data['ad_unit_id'].isin(
    train_data['ad_unit_id'].value_counts(normalize=True)[:-14].index)]

# viewable_impressions

In [ ]:
display_value_counts_and_norm(train_data['viewable_impressions'])

In [ ]:
train_data = train_data[train_data['viewable_impressions'].isin(
    train_data['viewable_impressions'].value_counts()[
        train_data['viewable_impressions'].value_counts() > 300].index)]

# measurable_impressions

In [ ]:
display_value_counts_and_norm(train_data['measurable_impressions'])

In [ ]:
train_data = train_data[train_data['measurable_impressions'].isin(
    train_data['measurable_impressions'].value_counts(normalize=True)[
        train_data['measurable_impressions'].value_counts(normalize=True) > 0.0005].index)]

# revenue_share_percent

In [ ]:
display_value_counts_and_norm(train_data['revenue_share_percent'])

In [ ]:
train_data.drop(['revenue_share_percent'], axis=1, inplace=True)
test_data.drop(['revenue_share_percent'], axis=1, inplace=True)

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
f, ax = plt.subplots(figsize=(16, 16))
ax = sns.heatmap(train_data.corr(), annot=True)

In [ ]:
f, ax = plt.subplots(figsize=(8, 8))
sns.scatterplot(data=train_data, x='measurable_impressions', y='total_impressions')

In [ ]:
train_data['is_CPM_positive'] = train_data['CPM'].apply(lambda x: int(bool(x)))

In [ ]:
train_data['is_CPM_positive'].value_counts(normalize=True)

In [ ]:
train_data.columns

In [ ]:
cat_features = ['site_id', 'ad_type_id', 'geo_id', 'device_category_id',
       'advertiser_id', 'order_id', 'line_item_type_id', 'os_id',
       'monetization_channel_id', 'ad_unit_id']

In [ ]:
%%time
for current_feature in cat_features:
    train_vals = list(train_data[current_feature].value_counts().index)
    test_vals = list(test_data[current_feature].value_counts().index)
    top_train_value = train_vals[0]
    for value in test_vals:
        if value not in train_vals:
            test_data[current_feature].replace({value: top_train_value}, inplace=True)

In [ ]:
display(train_data['line_item_type_id'].value_counts())
display(test_data['line_item_type_id'].value_counts())

In [ ]:
one_hot = OneHotEncoder()
one_hot.fit(train_data[cat_features])

In [ ]:
one_hot.transform(train_data[cat_features]).toarray()
one_hot.transform(test_data[cat_features]).toarray()

In [ ]:
num_features = ['date', 'total_impressions', 'viewable_impressions', 'measurable_impressions']

In [ ]:
train_data[num_features].shape, pd.DataFrame(one_hot.transform(train_data[cat_features]).toarray()).shape

In [ ]:
%%time
BIG_TRAIN_FULL_FOR_CLASS = pd.concat([train_data[num_features],
                                      pd.DataFrame(one_hot.transform(train_data[cat_features]).toarray(),
                                                  index = train_data[num_features].index)],
                                     axis=1)
print(BIG_TRAIN_FULL_FOR_CLASS.shape)
BIG_TEST_FULL_FOR_CLASS = pd.concat([test_data[num_features],
                                      pd.DataFrame(one_hot.transform(test_data[cat_features]).toarray(),
                                                  index = test_data[num_features].index)],
                                    axis=1)
print(BIG_TEST_FULL_FOR_CLASS.shape)

# пробуем бейзлайн  
сперва отделим нулевые от ненулевых значений, а потом предскажем на ненулевых  
который и оказался удачным

In [ ]:
num_features

In [ ]:
def scaler(data):
    return (data - data.mean()) / data.std()

In [ ]:
%%time
catboost_class = CatBoostClassifier(logging_level='Silent', random_seed=13)
catboost_class.fit(BIG_TRAIN_FULL_FOR_CLASS, train_data['is_CPM_positive'])

In [ ]:
y_class_pred = catboost_class.predict(BIG_TEST_FULL_FOR_CLASS)

In [ ]:
BIG_TRAIN_FULL_FOR_CLASS.shape, train_data.shape

In [ ]:
BIG_TRAIN_FULL_FOR_REGR = BIG_TRAIN_FULL_FOR_CLASS
BIG_TRAIN_FULL_FOR_REGR['is_CPM_positive'] = train_data['is_CPM_positive']
BIG_TRAIN_FULL_FOR_REGR = BIG_TRAIN_FULL_FOR_REGR[BIG_TRAIN_FULL_FOR_REGR['is_CPM_positive'] == 1]
Y_TRAIN_FOR_REGR = train_data[train_data['is_CPM_positive'] == 1]['CPM']

In [ ]:
%%time
catboost_regr = CatBoostRegressor(logging_level='Silent', random_seed=133, loss_function='RMSE',
                                 eval_metric='RMSE', iterations=1337)

catboost_regr.fit(BIG_TRAIN_FULL_FOR_REGR, Y_TRAIN_FOR_REGR)

In [ ]:
BIG_TEST_FULL_FOR_REGR = BIG_TEST_FULL_FOR_CLASS
BIG_TEST_FULL_FOR_REGR['is_CPM_positive'] = y_class_pred
BIG_TEST_FULL_FOR_REGR = BIG_TEST_FULL_FOR_REGR[BIG_TEST_FULL_FOR_REGR['is_CPM_positive'] == 1]
y_regr_pred = catboost_regr.predict(BIG_TEST_FULL_FOR_REGR)

In [ ]:
finish_pred = pd.DataFrame(y_class_pred)
finish_pred[finish_pred[0] == 1] = y_regr_pred
finish_pred

In [ ]:
%%time
for i in range(finish_pred.shape[0]):
    if finish_pred.iloc[i, 0] < 0:
        finish_pred.iloc[i, 0] = 0

In [ ]:
mean_squared_error(finish_pred, total_target)

In [ ]:
3218.7478511106797